# Introduction to pyNS 
pyNS is a Python library to programmatically access the Neuroscout API. pyNS let's you query the API and create analyses, without having to mess around with buildling any JSON requests yourself.

In this tutorial, I'll demonstrate how to query the API to create your own analysis.

In [28]:
from pyns import Neuroscout
api = Neuroscout('user@university.edu', 'yourpassword')

The `Neuroscout` object will be your main entry point to the API. You can instantiate this object without any credentials to access public API routes, or preferably with your Neuroscout credentials to be able to create and save your analyses. The `Neuroscout` object has links to each main API route, and each of these links implements the standard HTTP verbs that are suppoted by each route, such as `datasets`, `runs`, `predictors`, etc...

### Querying datasets and runs

In [3]:
api.datasets.get()

[{'description': {'Authors': ['Zadbood, A., Chen, J., Leong, Y.C., Norman, K.A., & Hasson, U.'],
   'BIDSVersion': '1.0.2',
   'Funding': 'National Institutes of Health (1R01MH112357-01 and 1R01MH112566-01)',
   'Name': 'Sherlock_Merlin',
   'ReferencesAndLinks': ['https://academic.oup.com/cercor/article/doi/10.1093/cercor/bhx202/4080827/How-We-Transmit-Memories-to-Other-Brains']},
  'id': 5,
  'mimetypes': ['audio/x-wav', 'image/png', 'text/plain', 'video/mp4'],
  'name': 'SherlockMerlin',
  'runs': [126,
   127,
   128,
   129,
   117,
   118,
   119,
   120,
   121,
   122,
   124,
   125,
   130,
   131,
   132,
   133,
   134],
  'summary': 'Participants watched Sherlock movie and listened to the audio recording of Merlin movie.',
  'tasks': [{'id': 4, 'name': 'MerlinMovie'}]},
 {'description': {'Acknowledgments': 'We thank Jason Gors, Kelsey G. Wheeler J. Swaroop Guntupalli, Matteo Visconti di Oleggio Castello, M. Ida Gobbini, Terry Sacket, and the rest of the DBIC (Dartmouth Bra

This request returns a list of two datasets, with information about the dataset, as well as the run IDs associated with this dataset. Let's focus on the first dataset, `Sherlock_Merlin`

If we want more information on the specific runs within this dataset, we can query the `runs` route, using the dataset_id associated with `Sherlock_Merlin`

In [4]:
dataset = api.datasets.get()[0]
api.runs.get(dataset_id=dataset['id'])

[{'acquisition': None,
  'dataset_id': 5,
  'duration': 1543.5,
  'id': 126,
  'number': None,
  'session': None,
  'subject': '28',
  'task': {'id': 4, 'name': 'MerlinMovie'}},
 {'acquisition': None,
  'dataset_id': 5,
  'duration': 1540.5,
  'id': 127,
  'number': None,
  'session': None,
  'subject': '29',
  'task': {'id': 4, 'name': 'MerlinMovie'}},
 {'acquisition': None,
  'dataset_id': 5,
  'duration': 1543.5,
  'id': 128,
  'number': None,
  'session': None,
  'subject': '30',
  'task': {'id': 4, 'name': 'MerlinMovie'}},
 {'acquisition': None,
  'dataset_id': 5,
  'duration': 1539.0,
  'id': 129,
  'number': None,
  'session': None,
  'subject': '31',
  'task': {'id': 4, 'name': 'MerlinMovie'}},
 {'acquisition': None,
  'dataset_id': 5,
  'duration': 1545.0,
  'id': 117,
  'number': None,
  'session': None,
  'subject': '19',
  'task': {'id': 4, 'name': 'MerlinMovie'}},
 {'acquisition': None,
  'dataset_id': 5,
  'duration': 1546.5,
  'id': 118,
  'number': None,
  'session': No

Great. In the analysis I'm buidling, I only want to focus on subjects '28' and '29', so lets extract those IDs:

In [5]:
run_ids = [r['id'] for r in api.runs.get(dataset_id=dataset['id']) if r['subject'] in ['28', '29']]
run_ids

[126, 127]

### Querying predictors

Now let's take a look at the predictors associated with these runs:

In [6]:
api.predictors.get(run_id=run_ids)

[{'description': 'Face detection confidence',
  'extracted_feature': {'created_at': '2018-04-19 19:43:02.103809',
   'description': 'Face detection confidence',
   'extractor_name': 'GoogleVisionAPIFaceExtractor',
   'id': 220,
   'modality': None},
  'id': 354,
  'max': 0.999985,
  'mean': 0.8007437250667854,
  'min': 0.5051182,
  'name': 'face_detectionConfidence',
  'num_na': 0,
  'source': 'extracted',
  'stddev': 0.17035985190329545},
 {'description': 'Face landmarking confidence',
  'extracted_feature': {'created_at': '2018-04-19 19:43:02.113270',
   'description': 'Face landmarking confidence',
   'extractor_name': 'GoogleVisionAPIFaceExtractor',
   'id': 221,
   'modality': None},
  'id': 355,
  'max': 0.8566273,
  'mean': 0.4601388381104185,
  'min': 0.10028671,
  'name': 'face_landmarkingConfidence',
  'num_na': 0,
  'source': 'extracted',
  'stddev': 0.18337511525219882},
 {'description': 'Noise components estimated using component based noise correction method',
  'id': 313

A bunch of useful information to help me choose some features! Let's keep it simple and go with 'face_detectionConfidence' and 'FramewiseDisplacement':

In [7]:
pred_ids = [r['id'] for r in api.predictors.get(run_id=run_ids) if r['name'] in \
            ['face_detectionConfidence', 'FramewiseDisplacement']]
pred_ids

[354, 308]

## Building an Analysis

Now, let's build an analysis. For this, we can use the `Analysis` class, which makes it easy to build an Analysis locally, by mirroring the Analysis representation on the server

In [8]:
analysis = api.analyses.create_analysis(dataset_id=5, name='My new analysis!', predictors=pred_ids, runs=run_ids)

`create_analysis` returns an object, which has been registered to the API. We can tell, because analyses are assigned a unique id

In [9]:
analysis.hash_id

'MaO1w'

In [10]:
# Some properties are read-only and came from the server
analysis.created_at

'2018-09-06T23:5'

We can edit this Analysis object to fill in any other Analysis details, and push them to the Neuroscout API:

In [11]:
analysis.description = "This is my analysis, and it's probably the best"
analysis.push()

Great. Now let's add a BIDS-Model to represent the analysis we want to execute

In [12]:
analysis.model = {
    "blocks": [
      {
        "auto_contrasts": True,
        "contrasts": [],
        "level": "run",
        "model": {
          "HRF_variables": [
            "face_detectionConfidence"
          ],
          "variables": [
            "face_detectionConfidence",
            "FramewiseDisplacement"
          ]
        },
        "transformations": []
      },
      {
        "auto_contrasts": True,
        "level": "dataset"
      }
    ],
    "input": {
      "subject": [
        "28",
        "29"
      ],
      "task": "MerlinMovie"
    },
}

In [16]:
analysis.push()
analysis.pull()
analysis.model

{'blocks': [{'auto_contrasts': True,
   'contrasts': [],
   'level': 'run',
   'model': {'HRF_variables': ['face_detectionConfidence'],
    'variables': ['face_detectionConfidence', 'FramewiseDisplacement']},
   'transformations': []},
  {'auto_contrasts': True, 'level': 'dataset'}],
 'input': {'subject': ['28', '29'], 'task': 'MerlinMovie'}}

### Compiling the analysis
Now that we have created and design an analysis we can ask the Neuroscout API to compile and verify this is a valid analysis.

In [17]:
analysis.compile()

{'TR': 1.5,
 'compile_traceback': '',
 'compiled_at': None,
 'created_at': '2018-09-06T23:5',
 'dataset_id': 5,
 'description': "This is my analysis, and it's probably the best",
 'hash_id': 'MaO1w',
 'model': {'blocks': [{'auto_contrasts': True,
    'contrasts': [],
    'level': 'run',
    'model': {'HRF_variables': ['face_detectionConfidence'],
     'variables': ['face_detectionConfidence', 'FramewiseDisplacement']},
    'transformations': []},
   {'auto_contrasts': True, 'level': 'dataset'}],
  'input': {'subject': ['28', '29'], 'task': 'MerlinMovie'}},
 'modified_at': '2018-09-06T23:5',
 'name': 'My new analysis!',
 'parent_id': None,
 'predictions': '',
 'predictors': [354, 308],
 'private': True,
 'runs': [126, 127],
 'status': 'PENDING',
 'task_name': 'MerlinMovie'}

In [18]:
analysis.get_status()

{'compile_traceback': '', 'status': 'PASSED'}

Great! Our analysis passed with no errors. We can now run our analysis using the `neuroscout-cli`. 

For more information on the `neuroscout-cli`, see here: https://github.com/neuroscout/neuroscout-cli

### Cloning our analysis

Now that we've gone off and run our analysis, we realized we want to make some changes. In this case, I want to run this analysis with more subjects. 

With Neuroscout this is easy, because I simply clone my previous analysis, and take off from I left off

In [23]:
new_analysis = analysis.clone()
new_analysis.hash_id
new_analysis.runs = [r['id'] for r in api.runs.get(dataset_id=dataset['id'])]
new_analysis.runs

[126,
 127,
 128,
 129,
 117,
 118,
 119,
 120,
 121,
 122,
 124,
 125,
 130,
 131,
 132,
 133,
 134]

In [24]:
new_analysis.push()
new_analysis.compile()

{'TR': 1.5,
 'compile_traceback': '',
 'compiled_at': None,
 'created_at': '2018-09-06T23:5',
 'dataset_id': 5,
 'description': "This is my analysis, and it's probably the best",
 'hash_id': 'A27pM',
 'model': {'blocks': [{'auto_contrasts': True,
    'contrasts': [],
    'level': 'run',
    'model': {'HRF_variables': ['face_detectionConfidence'],
     'variables': ['face_detectionConfidence', 'FramewiseDisplacement']},
    'transformations': []},
   {'auto_contrasts': True, 'level': 'dataset'}],
  'input': {'subject': ['28', '29'], 'task': 'MerlinMovie'}},
 'modified_at': '2018-09-06T23:5',
 'name': 'My new analysis!',
 'parent_id': None,
 'predictions': '',
 'predictors': [354, 308],
 'private': True,
 'runs': [126,
  127,
  128,
  129,
  117,
  118,
  119,
  120,
  121,
  122,
  124,
  125,
  130,
  131,
  132,
  133,
  134],
 'status': 'PENDING',
 'task_name': 'MerlinMovie'}

In [26]:
new_analysis.get_status()

{'compile_traceback': '', 'status': 'PASSED'}

Great, since this analysis also passed, I can easily run this forked version using the `neuroscout-cli`. If you have any questions on how to use the Neuroscout API, please see the API's Swagger Documentation, which covers all possible routes and parameters: https://alpha.neuroscout.org/swagger-ui